### optie 2

In [ ]:
import cv2
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time
import queue
import datetime
import pyttsx3

# Hoofdmap waar alle datasets en modellen staan
BASE_DIR = "datasets"

# 📷 Open de camera
cap = cv2.VideoCapture(0)
lock = Lock()

# OpenCV haarcascade gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# **📂 Automatisch alle modellen laden (LBPH)**
models = {}
for person in os.listdir(BASE_DIR):
    model_path = os.path.join(BASE_DIR, person, f"face_model_{person}.yml")
    if os.path.exists(model_path):
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read(model_path)
        models[person] = model
        print(f"✅ Model geladen voor {person}")

if not models:
    print("⚠️ Geen modellen correct geladen. Train eerst een model.")

# **🔹 Widgets**
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
threshold_slider = widgets.IntSlider(value=100, min=50, max=150, step=1, description="Threshold")
output = widgets.Output()

# **📌 Weergave in Jupyter Notebook**
display(threshold_slider, image_widget, stop_button, output)

# **🌍 Variabelen**
running = True

# Spraakfunctionaliteit en begroeting
speech_queue = queue.Queue()
greeting_delay = 10  # Aantal seconden voordat iemand opnieuw wordt begroet

# Dictionaries voor begroeting en last seen
greeted = {}    # {persoon: timestamp van laatste begroeting}
last_seen = {}  # {persoon: timestamp van laatste zien}

# In deze versie initialiseren we de TTS-engine niet globaal, maar per spraakbericht

def speech_worker():
    while running:
        try:
            text = speech_queue.get(timeout=1)
            # Maak voor elk bericht een nieuwe engine aan
            engine = pyttsx3.init()
            engine.say(text)
            engine.runAndWait()
            engine.stop()
        except queue.Empty:
            continue

# Start de spraak-thread
speech_thread = Thread(target=speech_worker, daemon=True)
speech_thread.start()

def get_greeting():
    now = datetime.datetime.now()
    hour = now.hour
    if 5 <= hour < 12:
        return "Goodmorning"
    elif 12 <= hour < 18:
        return "Goodafternoon"
    elif 18 <= hour < 22:
        return "Goodevening"
    else:
        return "Goodnight"

def greet_person(person_name):
    greeting = get_greeting()
    speech_message = f"{greeting}, hallo {person_name}, welcome on the digital platform Utrecht!"
    speech_queue.put(speech_message)
    with output:
        print(f"Begroeting voor {person_name}: {speech_message}")

def update_stream():
    global running, last_seen, greeted
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            with output:
                print("⚠️ Geen frame ontvangen. Controleer de camera.")
            break
        
        # Converteer naar grijswaarden
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        
        current_time = time.time()
        detected_names = set()
        
        # Verwerk elk gedetecteerd gezicht
        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (150, 150))
            
            best_name = "Unknown"
            best_confidence = float("inf")
            confidence_threshold = threshold_slider.value
            
            # Vergelijk met elk model
            for name, model in models.items():
                label, confidence = model.predict(face_resized)
                with output:
                    print(f"🔍 Model {name}: Confidence {confidence}")
                if confidence < best_confidence:
                    best_confidence = confidence
                    best_name = name
            
            # Als het gezicht zeker genoeg is (confidence lager dan de threshold)
            if best_confidence < confidence_threshold:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, best_name, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                with output:
                    print(f"✅ Herkend als {best_name} met confidence {best_confidence}")
                detected_names.add(best_name)
            else:
                with output:
                    print(f"❌ Gezicht niet zeker genoeg: {best_name} ({best_confidence})")
        
        # Voor alle gedetecteerde namen: update last_seen en trigger begroeting indien nodig
        for name in detected_names:
            last_seen[name] = current_time
            if name not in greeted or (current_time - greeted.get(name, 0)) >= greeting_delay:
                greet_person(name)
                greeted[name] = current_time
        
        # Voor personen die niet in dit frame zijn gedetecteerd: 
        # Als ze langer dan greeting_delay niet zijn gezien, verwijder ze uit greeted zodat ze later weer begroet worden.
        for name in list(greeted.keys()):
            if name not in detected_names and (current_time - last_seen.get(name, 0)) > greeting_delay:
                with output:
                    print(f"Reset greeting voor {name}")
                del greeted[name]
        
        # Update het live beeld
        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        
        time.sleep(0.03)

def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()
    with output:
        print("Stream gestopt.")

stop_button.on_click(stop_stream)

# Start de live stream in een aparte thread
thread = Thread(target=update_stream, daemon=True)
thread.start()


### optie 3 (CNN)

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time

# Modelpad instellen
MODEL_PATH = "face_recognition_modelv2.h5"

# Laad het getrainde model
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("Geen model gevonden. Train eerst een model met een dataset.")

model = load_model(MODEL_PATH)
print("Model geladen")

# Hoofdmap voor datasets
BASE_DIR = "datasets"

# Open de camera
cap = cv2.VideoCapture(1)
lock = Lock()

# OpenCV haarcascade voor gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Labels inladen
label_dict = {}
idx = 0  # Unieke index toewijzen aan elk label
for person in os.listdir(BASE_DIR):
    person_path = os.path.join(BASE_DIR, person, "processed_faces")
    if os.path.isdir(person_path):
        label_dict[idx] = person
        idx += 1

if not label_dict:
    raise ValueError("Geen dataset gevonden. Train eerst een model.")

print(f"Labels geladen: {label_dict}")

# Widgets
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
threshold_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description="Threshold")
output = widgets.Output()

# Weergave in Jupyter Notebook
display(threshold_slider, image_widget, stop_button, output)

# Variabelen
running = True

# Live gezichtsdetectie en herkenning
def update_stream():
    global running
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            print("Geen frame ontvangen. Controleer de camera.")
            break

        # ✅ Stap 1: Converteer naar grijswaarden
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (100, 100))

            # ✅ Stap 2: Fix - Zet om naar RGB formaat (3 kanalen)
            face_resized = np.expand_dims(face_resized, axis=-1)  # Maak van (100, 100) -> (100, 100, 1)
            face_resized = np.repeat(face_resized, 3, axis=-1)  # Zet (100,100,1) om naar (100,100,3)
            face_resized = np.expand_dims(face_resized, axis=0)  # Maak het 4D voor modelinvoer
            face_resized = face_resized.astype("float32") / 255.0  # Normaliseren

            # Haal de actuele threshold op uit de slider
            confidence_threshold = threshold_slider.value

            # ✅ Stap 3: Laat het model een voorspelling doen
            predictions = model.predict(face_resized)[0]
            best_idx = np.argmax(predictions)
            confidence = predictions[best_idx]

            if confidence > confidence_threshold:
                best_name = label_dict.get(best_idx, "Unknown")
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{best_name} ({confidence:.2f})", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                print(f"Herkend als {best_name} met confidence {confidence:.2f}")
            else:
                print(f"Onzeker gezicht: {confidence:.2f}")

        # ✅ Stap 4: Toon live beeld in Jupyter
        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        
        time.sleep(0.03)


# Stop de live stream
def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()

stop_button.on_click(stop_stream)

# Start de live stream in een aparte thread
thread = Thread(target=update_stream, daemon=True)
thread.start()


Model geladen
Labels geladen: {0: 'Arend', 1: 'Bobby', 2: 'PJ', 3: 'Robert', 4: 'Unknown'}


FloatSlider(value=0.5, description='Threshold', max=1.0, step=0.01)

Image(value=b'', format='jpeg')

Button(description='Stop', style=ButtonStyle())

Output()

1/1 [==============================] - 0s 156ms/step
Herkend als PJ met confidence 0.99
1/1 [==============================] - 0s 33ms/step
Herkend als PJ met confidence 0.54
1/1 [==============================] - 0s 29ms/step
Herkend als PJ met confidence 0.56
1/1 [==============================] - 0s 34ms/step
Herkend als PJ met confidence 0.54
1/1 [==============================] - 0s 25ms/step
Herkend als PJ met confidence 0.52
1/1 [==============================] - 0s 25ms/step
Herkend als PJ met confidence 0.55
1/1 [==============================] - 0s 29ms/step
Herkend als PJ met confidence 0.51
1/1 [==============================] - 0s 35ms/step
Onzeker gezicht: 0.50
1/1 [==============================] - 0s 23ms/step
Onzeker gezicht: 0.48
1/1 [==============================] - 0s 28ms/step
Onzeker gezicht: 0.48
1/1 [==============================] - 0s 22ms/step
Herkend als Bobby met confidence 0.50
1/1 [==============================] - 0s 24ms/step
Onzeker gezicht: 0.48
1/1 

### optie 4

In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time

# Model en labels inladen
MODEL_PATH = "face_recognition_modelv2.h5"
LABEL_PATH = "label_mapping.npy"

if not os.path.exists(MODEL_PATH) or not os.path.exists(LABEL_PATH):
    raise FileNotFoundError("Geen model of labels gevonden. Train eerst een model.")

model = load_model(MODEL_PATH, compile=False)  # Laden zonder automatische compilatie
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])  # Handmatige compilatie

label_dict = np.load(LABEL_PATH, allow_pickle=True).item()
print("✅ Model en labels geladen:", label_dict)

cap = cv2.VideoCapture(0)
lock = Lock()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Widgets
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
threshold_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description="Threshold")
output = widgets.Output()

display(threshold_slider, image_widget, stop_button, output)

running = True

def update_stream():
    global running
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            print("Geen frame ontvangen. Controleer de camera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (100, 100))
            face_resized = np.expand_dims(face_resized, axis=0)  
            face_resized = face_resized.astype("float32") / 255.0  

            confidence_threshold = threshold_slider.value

            predictions = model.predict(face_resized)[0]
            best_idx = np.argmax(predictions)
            confidence = predictions[best_idx]

            if confidence > confidence_threshold:
                best_name = label_dict.get(best_idx, "Unknown")
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{best_name} ({confidence:.2f})", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        
        time.sleep(0.03)

stop_button.on_click(lambda _: setattr(running, False))

thread = Thread(target=update_stream, daemon=True)
thread.start()


✅ Model en labels geladen: {'Arend': 0, 'Bobby': 1, 'PJ': 2, 'Robert': 3, 'Unknown': 4}


FloatSlider(value=0.5, description='Threshold', max=1.0, step=0.01)

Image(value=b'', format='jpeg')

Button(description='Stop', style=ButtonStyle())

Output()

1/1 [==============================] - 0s 30ms/step


### optie 5

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import queue
import time
import datetime
import pyttsx3

# Paden voor model en labels
MODEL_PATH = "face_recognition_modelv2.h5"
LABEL_PATH = "label_mapping.npy"

# Controleer of het model en de labels bestaan
if not os.path.exists(MODEL_PATH) or not os.path.exists(LABEL_PATH):
    raise FileNotFoundError("Geen model of labels gevonden. Train eerst een model.")

# Laad het model en de labelmapping (verwacht mapping: label_index -> persoonsnaam)
model = load_model(MODEL_PATH, compile=False)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
label_dict = np.load(LABEL_PATH, allow_pickle=True).item()
print("✅ Model en labels geladen:", label_dict)

# Start de camera
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise Exception("Camera niet beschikbaar.")

# Laad de Haarcascade voor gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Widgets voor Jupyter Notebook
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
output = widgets.Output()
display(image_widget, stop_button, output)

# Globale variabelen en threading objecten
running = True
frame_queue = queue.Queue(maxsize=5)
lock = Lock()

# Constante voor minimale vertrouwen (confidence) en gap
CONFIDENCE_THRESHOLD = 0.5
MIN_CONFIDENCE_GAP = 0.1

# Spraakfunctionaliteit: queue en timers
speech_queue = queue.Queue()
greeting_timers = {}  # Houdt per persoon bij wanneer de laatste begroeting is gebeurd
greeting_delay = 10   # Aantal seconden tussen begroetingen voor dezelfde persoon

# Houdt bij wanneer een persoon voor het laatst is gezien
last_seen = {}

# Initialiseer pyttsx3 spraakengine
speech_engine = pyttsx3.init()

def speech_worker():
    while running:
        try:
            text = speech_queue.get(timeout=1)
            speech_engine.say(text)
            speech_engine.runAndWait()
        except queue.Empty:
            continue

# Start de spraak-thread
speech_thread = Thread(target=speech_worker, daemon=True)
speech_thread.start()

def get_greeting():
    now = datetime.datetime.now()
    hour = now.hour
    if 5 <= hour < 12:
        return "Goodmorning"
    elif 12 <= hour < 18:
        return "Goodafternoon"
    elif 18 <= hour < 22:
        return "Goodevening"
    else:
        return "Goodnight"

def check_greeting(person_name):
    current_time = time.time()
    # Als de persoon niet recent is begroet, trigger de spraakbericht
    if person_name not in greeting_timers or (current_time - greeting_timers[person_name]) >= greeting_delay:
        greeting = get_greeting()
        speech_message = f"{greeting}, hallo {person_name}, welcome on the digital platform Utrecht!"
        speech_queue.put(speech_message)
        greeting_timers[person_name] = current_time
        with output:
            print(f"Begroeting voor {person_name}: {speech_message}")

def capture_frames():
    global running
    while running:
        ret, frame = cap.read()
        if ret:
            if not frame_queue.full():
                frame_queue.put(frame)
        else:
            with output:
                print("Geen frame ontvangen. Controleer de camera.")
            break
        time.sleep(0.01)

def process_frames():
    global running
    while running:
        if not frame_queue.empty():
            frame = frame_queue.get()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
            
            current_time = time.time()
            if len(faces) == 0:
                # Geen gezichten gedetecteerd: clear last_seen
                last_seen.clear()
            else:
                for (x, y, w, h) in faces:
                    face_img = frame[y:y+h, x:x+w]
                    face_resized = cv2.resize(face_img, (100, 100))
                    face_resized = np.expand_dims(face_resized, axis=0)
                    face_resized = face_resized.astype("float32") / 255.0

                    predictions = model.predict(face_resized)[0]
                    sorted_idx = np.argsort(predictions)[::-1]
                    best_idx = sorted_idx[0]
                    second_best_idx = sorted_idx[1]
                    confidence = predictions[best_idx]
                    gap = predictions[best_idx] - predictions[second_best_idx]
                    
                    with output:
                        print(f"Voorspellingen: {predictions}, best: {best_idx} ({confidence:.2f}), gap: {gap:.2f}")
                    
                    if confidence > CONFIDENCE_THRESHOLD and gap > MIN_CONFIDENCE_GAP:
                        best_name = label_dict.get(best_idx, "Unknown")
                        if best_name != "Unknown":
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                            cv2.putText(frame, f"{best_name} ({confidence:.2f})", (x, y-10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                            last_seen[best_name] = current_time
                            check_greeting(best_name)
                            
            _, buffer = cv2.imencode('.jpg', frame)
            image_widget.value = buffer.tobytes()
        else:
            time.sleep(0.01)

def greeting_timer_manager():
    """Controleert elke seconde of een persoon langer dan greeting_delay niet is gezien.
       Zo worden de timers gereset en kan bij een nieuwe verschijning opnieuw begroet worden."""
    while running:
        current_time = time.time()
        for person in list(greeting_timers.keys()):
            # Als iemand niet meer voorkomt in last_seen of langer dan greeting_delay niet is gezien:
            if person not in last_seen or (current_time - last_seen[person]) >= greeting_delay:
                with output:
                    print(f"Reset timer voor {person}")
                del greeting_timers[person]
        time.sleep(1)

def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()
    with output:
        print("Stream gestopt.")

stop_button.on_click(stop_stream)

# Start de threads voor framecaptatie, verwerking en timerbeheer
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frames, daemon=True)
timer_thread = Thread(target=greeting_timer_manager, daemon=True)
capture_thread.start()
process_thread.start()
timer_thread.start()


✅ Model en labels geladen: {'Arend': 0, 'Bobby': 1, 'PJ': 2, 'Robert': 3, 'Unknown': 4}


Image(value=b'', format='jpeg')

Button(description='Stop', style=ButtonStyle())

Output()

1/1 [==============================] - 0s 157ms/step
Voorspellingen: [9.2640406e-10 1.9870993e-05 9.9997962e-01 5.1718763e-10 4.8399443e-07], best: 2 (1.00), gap: 1.00
1/1 [==============================] - 0s 35ms/step
Voorspellingen: [1.5219286e-09 3.3888180e-05 9.9996567e-01 8.5901841e-10 4.8437738e-07], best: 2 (1.00), gap: 1.00
1/1 [==============================] - 0s 35ms/step
Voorspellingen: [1.5070099e-09 2.9317840e-05 9.9997008e-01 4.7784798e-10 6.4923802e-07], best: 2 (1.00), gap: 1.00
1/1 [==============================] - 0s 32ms/step
Voorspellingen: [1.2537987e-09 3.0591589e-05 9.9996901e-01 1.0746034e-09 3.6356383e-07], best: 2 (1.00), gap: 1.00
1/1 [==============================] - 0s 52ms/step
Voorspellingen: [1.3593123e-09 2.8604107e-05 9.9997091e-01 7.5254580e-10 4.7596347e-07], best: 2 (1.00), gap: 1.00
1/1 [==============================] - 0s 38ms/step
Voorspellingen: [1.0381607e-09 2.7880538e-05 9.9997187e-01 1.5374657e-09 2.7139683e-07], best: 2 (1.00), gap: 1.

### optie 6

In [ ]:
import cv2
import os
import numpy as np
import pyttsx3
import queue
from threading import Thread, Lock
import time
import datetime
import ipywidgets as widgets
from IPython.display import display

# Hoofdmap waar alle datasets en modellen staan
BASE_DIR = "datasets"

# RTSP-camera instellen
cap = cv2.VideoCapture(1)
lock = Lock()

# OpenCV haarcascade gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Automatisch alle modellen laden
models = {}
for person in os.listdir(BASE_DIR):
    model_path = os.path.join(BASE_DIR, person, f"face_model_{person}.yml")
    if os.path.exists(model_path):
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read(model_path)
        models[person] = model
        print(f"✅ Model geladen voor {person}")

if not models:
    print("⚠️ Geen modellen correct geladen. Train eerst een model.")

# Widgets voor Jupyter Notebook
image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
output = widgets.Output()
display(image_widget, stop_button, output)

# Variabelen
running = True
speech_queue = queue.Queue()
recognized_people = set()
greeting_timers = {}  # Dictionary per persoon om begroetingstimers bij te houden
greeting_delay = 10  # Wacht 5 seconden per persoon voordat hij opnieuw begroet wordt

# **Spraakfunctie** (geïnitialiseerd in de hoofdthread)
speech_engine = pyttsx3.init()

def speech_worker():
    while running:
        try:
            text = speech_queue.get(timeout=1)
            speech_engine.say(text)
            speech_engine.runAndWait()
        except queue.Empty:
            continue

# Start de spraak-thread
speech_thread = Thread(target=speech_worker, daemon=True)
speech_thread.start()

def get_greeting():
    now = datetime.datetime.now()
    hour = now.hour
    time_str = now.strftime("%H:%M")  # Tijd als hh:mm

    if 5 <= hour < 12:
        return f"Good morning!"
    elif 12 <= hour < 18:
        return f"Good afternoon!"
    elif 18 <= hour < 22:
        return f"Good evening!"
    else:
        return f"Good night!"

def check_greeting(person_name):
    """ Controleert of deze specifieke persoon al begroet is en start een nieuwe begroeting als nodig """
    global greeting_timers
    current_time = time.time()

    # Als er nog geen timer is voor deze persoon, of als het meer dan 10 seconden geleden is
    if person_name not in greeting_timers or (current_time - greeting_timers[person_name]) >= greeting_delay:
        greeting = get_greeting()
        speech_queue.put(f"{greeting} Hello {person_name}, welcome home!")
        
        # Reset de timer per persoon
        greeting_timers[person_name] = current_time

# **Live stream & herkenning**
def update_stream():
    global running
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            print("⚠️ Geen frame ontvangen. Controleer de camera.")
            break

        # Converteer frame naar grijswaarden
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        if len(faces) == 0:
            print("ℹ️ Geen gezichten gedetecteerd.")
            recognized_people.clear()  # Reset de lijst als niemand meer wordt gezien
            continue  # Ga direct door naar de volgende iteratie

        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (150, 150))  # Test met 150x150

            best_name = "Unknown"
            best_confidence = float("inf")
            second_best_confidence = float("inf")  # Tweede beste match
            
            for name, model in models.items():
                label, confidence = model.predict(face_resized)
                print(f"🔍 Model {name}: Confidence {confidence} (Lager is beter)")
                
                if confidence < best_confidence:
                    second_best_confidence = best_confidence  # Sla oude beste op
                    best_confidence = confidence
                    best_name = name
                elif confidence < second_best_confidence:
                    second_best_confidence = confidence  # Tweede beste match
            
            # Controleer of het verschil groot genoeg is tussen 1e en 2e match
            confidence_threshold = 130  # Hoe lager, hoe strenger
            min_confidence_gap = 5  # Minimale afstand tussen eerste en tweede match
            
            if best_confidence < confidence_threshold and (second_best_confidence - best_confidence) > min_confidence_gap:
                current_time = time.time()
                
                # Controleer of deze persoon lang genoeg niet gezien is voor een nieuwe begroeting
                if best_name not in greeting_timers or (current_time - greeting_timers[best_name]) >= greeting_delay:
                    recognized_people.add(best_name)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, best_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    print(f"✅ Gezicht herkend als {best_name} met confidence {best_confidence}")
                    
                    # Roep begroetingsfunctie aan
                    check_greeting(best_name)
            else:
                print(f"❌ Gezicht niet zeker genoeg herkend: {best_name} ({best_confidence}), tweede {second_best_confidence}")

        # Toon het beeld in Jupyter Notebook
        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        
        time.sleep(0.03)

# **Stop de live stream**
def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()

stop_button.on_click(stop_stream)

# **Start de live stream in een aparte thread**
thread = Thread(target=update_stream, daemon=True)
thread.start()
